# Simple End to End

This part of the tutorial demonstrates the basic end-to-end flow from an ONNX model to a generated hardware IP. We encourage you to run this example to try our automated toolflow, and then follow the rest of the tutorial which provides a more detailed explanation of our codebase.

In [1]:
import pathlib
from fpgaconvnet.parser.Parser import Parser
from fpgaconvnet.platform.Platform import Platform
from fpgaconvnet.optimiser.solvers import GreedyPartition

# load network
network_name = "single_layer"
onnx_path = f"./hardware-tutorial-assets/host-code/MNIST/{network_name}.onnx"
parser = Parser(custom_onnx=False, batch_size=1)
net = parser.onnx_to_fpgaconvnet(onnx_path)

device_path = "../../fpgaconvnet-optimiser/examples/platforms/zedboard.toml"
platform = Platform()
platform.update(device_path)
# We select port_width = 64 for solver in this example
platform.port_width   = 64

# Design Space Exploration
opt = GreedyPartition(net, platform)
opt.bram_to_lut = False
opt.off_chip_streaming = False
opt.balance_bram_uram = False
opt.rsc_allocation = 0.75
opt.transforms = []
opt.transforms_probs = []

opt.run_solver()
opt.update_partitions()

# export configuration and prediction report
pathlib.Path(network_name).mkdir(parents=True, exist_ok=True)
opt.create_report(f"{network_name}/report.json")
opt.net.save_all_partitions(f"{network_name}/config.json")

throughput = -opt.get_cost()
print(f"predicted throughput (img/s): {throughput}")
print(f"predicted resource usage: {net.partitions[0].get_resource_usage()}")
print(f"selected port_widthin solver: {platform.port_width} bits")

╔══════════════════════════════════════════════╦═════════╦══╦═══════════╦═══╦══════════════════════════════╦══════════╦══════════════════════════╦══════════╗
║ single partition (Part 1) cost (throughput): ║ 74515.6 ║  ║ slowdown: ║ 1 ║ partition optimisation time: ║ 0.06 sec ║ total optimisation time: ║ 0.06 sec ║
╚══════════════════════════════════════════════╩═════════╩══╩═══════════╩═══╩══════════════════════════════╩══════════╩══════════════════════════╩══════════╝
| COST:                     |    | RESOURCES:   |            |                |                 |              |             |              |             |
|---------------------------|----|--------------|------------|----------------|-----------------|--------------|-------------|--------------|-------------|
|                           |    | BRAM         | DSP        | LUT            | FF              | BW           | BW_IN       | BW_OUT       | BW_WEIGHT   |
| 74515.648286 (throughput) |    | 45.00/280    | 100.00/2

In [2]:
import toml
from fpgaconvnet.hls.generate.network import GenerateNetwork

with open(device_path, "r") as f:
    device = toml.load(f) 
    device_name = device["device"]["part"]
    clock_period = 1000 / device["system"]["board_frequency"]
    # port_width = device["system"]["port_width"] 

# Similarly, select port_width = 64 for HLS 
port_width = 64

print(f"device name is {device_name}")
print(f"clock period is {clock_period}")
print(f"platform maximum port width is {port_width}")
    

# generate HLS code
# gen_net = GenerateNetwork(network_name, f"{network_name}/config.json", onnx_path, device_name)
gen_net = GenerateNetwork(network_name, f"{network_name}/config.json", onnx_path, device_name, clock_period, port_width)
gen_net.create_partition_project(0)

FIXME: use HLS backend in Parser

****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/2019.1/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/opt/Xilinx/2019.1/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'zy18' on host 'ubuntu' (Linux_x86_64 version 5.15.0-94-generic) on Mon Feb 19 00:19:28 GMT 2024
INFO: [HLS 200-10] On os Ubuntu 20.04.6 LTS
INFO: [HLS 200-10] In directory '/home/zy18/codeDev/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end'
Sourcing Tcl script '/home/zy18/codeDev/fpgaconvnet-tutorial/fpgaconvnet-hls/fpgaconvnet/hls/scripts/hls/create_partition_project.tcl'
project: partition_0
INFO: [HLS 200-10] Creating and opening project '/home/

In [3]:
import numpy as np
# generate random input image, you can replace it with a fixed image
input_image = np.random.rand(1,1,28,28).astype(np.float32)
# run C simulation
gen_net.run_testbench(0, input_image)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/2019.1/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/opt/Xilinx/2019.1/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'zy18' on host 'ubuntu' (Linux_x86_64 version 5.15.0-94-generic) on Mon Feb 19 00:19:34 GMT 2024
INFO: [HLS 200-10] On os Ubuntu 20.04.6 LTS
INFO: [HLS 200-10] In directory '/home/zy18/codeDev/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end'
Sourcing Tcl script '/home/zy18/codeDev/fpgaconvnet-tutorial/fpgaconvnet-hls/fpgaconvnet/hls/scripts/hls/run_csim.tcl'
INFO: [HLS 200-10] Opening project '/home/zy18/codeDev/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end/partition_0'.
INFO: 

In [4]:
# run C synthesis and export IP
gen_net.generate_partition_hardware(0)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/2019.1/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/opt/Xilinx/2019.1/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'zy18' on host 'ubuntu' (Linux_x86_64 version 5.15.0-94-generic) on Mon Feb 19 00:20:19 GMT 2024
INFO: [HLS 200-10] On os Ubuntu 20.04.6 LTS
INFO: [HLS 200-10] In directory '/home/zy18/codeDev/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end'
Sourcing Tcl script '/home/zy18/codeDev/fpgaconvnet-tutorial/fpgaconvnet-hls/fpgaconvnet/hls/scripts/hls/run_csynth.tcl'
INFO: [HLS 200-10] Opening project '/home/zy18/codeDev/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end/partition_0'.
INFO

INFO: [XFORM 203-712] Applying dataflow to function 'sliding_window<1u, 28u, 28u, 1u, 0u, 0u, 0u, 0u, 1u, 1u, 5u, 5u, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0> >', detected/extracted 2 process function(s): 
	 'sliding_window_line_shift<1u, 28u, 28u, 1u, 0u, 0u, 0u, 0u, 5u, 5u, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0> >'
	 'sliding_window_out<1u, 28u, 28u, 1u, 0u, 0u, 0u, 0u, 1u, 1u, 5u, 5u, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0> >'.
INFO: [XFORM 203-712] Applying dataflow to function 'conv<1u, 24u, 24u, 1u, 2u, 1u, 2u, 25u, 5u, 5u, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0>, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0>, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0> >', detected/extracted 3 process function(s): 
	 'conv_intr<1u, 24u, 24u, 1u, 2u, 1u, 2u, 25u, 5u, 5u, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0>, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0> >'
	 'conv_mul<1u, 24u, 24u, 1u, 2u, 1u, 25u, 5u, 5u, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0>, 

INFO: [XFORM 203-541] Flattening a loop nest 'Loop-1' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/conv.hpp:232:32) in function 'conv_intr<1u, 24u, 24u, 1u, 2u, 1u, 2u, 25u, 5u, 5u, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0>, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0> >343'.
INFO: [XFORM 203-541] Flattening a loop nest 'Loop-1' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/conv.hpp:232:32) in function 'conv_intr<1u, 24u, 24u, 1u, 2u, 1u, 2u, 25u, 5u, 5u, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0>, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0> >341'.
INFO: [XFORM 203-541] Flattening a loop nest 'Loop-1' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/conv.hpp:232:32) in function 'conv_intr<1u, 24u, 24u, 1u, 2u, 1u, 2u, 25u, 5u, 5u, ap_fixed<16, 8, (ap_q_mode)0, (ap_o_mode)3, 0>, ap_fixed<16, 8, (ap_q_mode)0, (

INFO: [ANALYSIS 214-51] Found 'WAR' intra dependency for variable 'line_buffer[0].V.V' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/sliding_window.hpp:267).
INFO: [ANALYSIS 214-51] Found 'WAR' intra dependency for variable 'line_buffer[1].V.V' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/sliding_window.hpp:267).
INFO: [ANALYSIS 214-51] Found 'WAR' intra dependency for variable 'line_buffer[2].V.V' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/sliding_window.hpp:267).
INFO: [ANALYSIS 214-51] Found 'WAR' intra dependency for variable 'line_buffer[3].V.V' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/sliding_window.hpp:267).
INFO: [ANALYSIS 214-51] Found 'WAR' intra dependency for variable 'window_buffer[0][0].' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/py

INFO: [ANALYSIS 214-51] Found 'WAR' intra dependency for variable 'frame_cache[4][0].V' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/sliding_window.hpp:277).
INFO: [ANALYSIS 214-51] Found 'WAR' intra dependency for variable 'frame_cache[4][1].V' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/sliding_window.hpp:277).
INFO: [ANALYSIS 214-51] Found 'WAR' intra dependency for variable 'frame_cache[4][2].V' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/sliding_window.hpp:277).
INFO: [ANALYSIS 214-51] Found 'WAR' intra dependency for variable 'frame_cache[4][3].V' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/sliding_window.hpp:277).
INFO: [ANALYSIS 214-51] Found 'WAR' intra dependency for variable 'frame_cache[4][4].V' (/home/alex/miniconda3/envs/fpgaconvnet-tutorial/lib

...

INFO: [RTMG 210-285] Implementing FIFO 'fork_out_0_6_0_2_U(fifo_w16_d2_A_x7)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'fork_out_0_6_0_3_U(fifo_w16_d2_A_x7)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'fork_out_0_6_0_4_U(fifo_w16_d2_A_x7)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'fork_out_0_6_1_0_U(fifo_w16_d2_A_x7)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'fork_out_0_6_1_1_U(fifo_w16_d2_A_x7)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'fork_out_0_6_1_2_U(fifo_w16_d2_A_x7)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'fork_out_0_6_1_3_U(fifo_w16_d2_A_x7)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'fork_out_0_6_1_4_U(fifo_w16_d2_A_x7)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'fork_out_0_6_2_0_U(fifo_w16_d2_A_x7)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'fork_out_0_6_2_1_U(fifo_w16_d2_A_x7)' using Shift R

INFO: [RTMG 210-285] Implementing FIFO 'cache_13_U(fifo_w16_d2_A_x8)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'cache_14_U(fifo_w16_d2_A_x8)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'cache_15_U(fifo_w16_d2_A_x8)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'start_for_squeeze_out_U0_U(start_for_squeeze_out_U0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'in_0_V_V_U(fifo_w16_d256_A_x)' using Block RAMs.
INFO: [RTMG 210-285] Implementing FIFO 'weights_reloading_in_3_U(fifo_w32_d5_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'out_hw_offset_c_U(fifo_w29_d5_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'single_layer_conv1_C_517_U(fifo_w16_d2_A_x9)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'single_layer_conv1_C_518_U(fifo_w16_d2_A_x9)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'single_layer_conv1_C_519_U(fifo_w16_d2_A_x9)' using Shif

INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_251_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_252_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_253_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_254_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_255_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_256_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_257_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: 

INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_314_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_315_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_99_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_98_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_97_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_96_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_95_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG

INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_38_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_36_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_35_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_34_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_32_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_31_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 210-278] Implementing memory 'Block_proc_single_layer_conv1_C_30_ram (RAM)' using distributed RAMs with power-on initialization.
INFO: [RTMG 2

In [5]:
gen_net.run_cosimulation(0, input_image)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/2019.1/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/opt/Xilinx/2019.1/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'zy18' on host 'ubuntu' (Linux_x86_64 version 5.15.0-94-generic) on Mon Feb 19 00:28:26 GMT 2024
INFO: [HLS 200-10] On os Ubuntu 20.04.6 LTS
INFO: [HLS 200-10] In directory '/home/zy18/codeDev/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end'
Sourcing Tcl script '/home/zy18/codeDev/fpgaconvnet-tutorial/fpgaconvnet-hls/fpgaconvnet/hls/scripts/hls/run_cosim.tcl'
INFO: [HLS 200-10] Opening project '/home/zy18/codeDev/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end/partition_0'.
INFO:

## Troubleshooting 
1. In `Module.py` and `Module3D.py` from `/fpgaconvnet-model/fpgaconvnet/models/modules/`, comment out the line:
   ```python
   # from xgboost import XGBRegressor
   ```
2. If the version of numpy cannot compile, you may need to uninstall numpy and reinstall a downgraded version. Version 2.0.0 is recommended. 
3. You may be working on a higher version of Linux OS that doesn't support Vivado 2019.1. In this case, Vivado may fail to simulate. Try deleting the ld folder at: `/Xilinx/Vivado/2019.1/tps/lnx64/binutils-2.26/bin/ld`
## What's next
Now that an ip has been generated, you may proceed to `hardware-tutorial.ipynb` to integrate the IP to a project. 